In this note book, we will practise how to segment and cluster neighborhoods in Toronto using Foursquare API

In [1]:
# use url to store the target website
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
! pip install beautifulsoup4
! pip install lxml

In [4]:
#create a object to handle the contents of the website
source = requests.get(url).text
soup = BeautifulSoup(source,'lxml')

In [5]:
#find the table in the website and convert it to panda dataframe
table = soup.find('table')
#print(table.prettify())
table_rows = table.find_all('tr')

canadadata = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text.rstrip() for i in td]
    canadadata.append(row)

canadadata = canadadata[1:]
#print(canadadata)

df_can = pd.DataFrame(canadadata, columns = ['Postal code', 'Borough', 'Neighborhood'])
#df_can = df_can.drop([0],axis = 0)
df_can.head(5)

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [6]:
#modify the dataframe to satisfy the request
#1. delete the rows with not assigned Borough
df_can = df_can[df_can.Borough != 'Not assigned']

#2. replace / with ,
df_can['Neighborhood']=df_can['Neighborhood'].str.replace('/', ',')

#3. assign neighborhood as the same as the borough
df_can.loc[df_can['Neighborhood']=='', 'Neighborhood'] = df_can.loc[df_can['Neighborhood']=='','Borough']

#4. reset the index
df_can = df_can.reset_index(drop = True)
df_can.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [8]:
# read the coordinate information from http://cocl.us/Geospatial_data
df_laglng = pd.read_csv('http://cocl.us/Geospatial_data')

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
df_can_joint = df_can.join(df_laglng.set_index('Postal Code'), on = 'Postal code')
df_can_joint.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


     |████████████████████████████████| 102kB 15.0MB/s ta 0:00:01


None
